In [1]:
using Laplacians

objc[69592]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[69592]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[69592]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[69592]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/xu_4/"

"../graphs/xu_4/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
# need to get proper weights for the tree
u,v,w = findnz(tree)
for i in 1:nnz(tree)
    tree[u[i],v[i]] = a[u[i],v[i]]
end

In [8]:
x = readFromFile(graphp * "x.vec");

In [9]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);
a.n

100000

In [10]:
@time F = treeSolver(tree)
@time F(b);

  0.727705 seconds (1.31 M allocations: 82.066 MB, 1.59% gc time)
  0.198876 seconds (1.72 M allocations: 35.537 MB, 4.69% gc time)


In [11]:
# a run on standard Float64 data types
numIts = 1000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 860.8801484022515
Working on iteration 20 with a-norm error 793.7383264752438
Working on iteration 30 with a-norm error 733.9814793246815
Working on iteration 40 with a-norm error 661.9337522373115
Working on iteration 50 with a-norm error 585.1691714109404
Working on iteration 60 with a-norm error 504.83259207189445
Working on iteration 70 with a-norm error 442.6859946955575
Working on iteration 80 with a-norm error 386.93309322534753
Working on iteration 90 with a-norm error 323.33291727946914
Working on iteration 100 with a-norm error 272.7082215745001
Working on iteration 110 with a-norm error 229.77955386945197
Working on iteration 120 with a-norm error 190.01325385463656
Working on iteration 130 with a-norm error 156.4185126908476
Working on iteration 140 with a-norm error 134.3042490417403
Working on iteration 150 with a-norm error 112.54865407676867
Working on iteration 160 with a-norm error 95.00009400973525
Working on iteration 170 wi

numIts = 1000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end